In [1]:
import numpy as np #importamos numpy para un mejor manejo de matrizes, filas y valores
import matplotlib.pyplot as plt #esta libreria la usaremos para graficar
from pyspark.sql import SparkSession #PySpark sera nuestra opcion para el manejo de grandes volumenes de datos
import os
import pandas

# Crea una sesión de Spark
spark = SparkSession.builder \
    .appName("MiAplicacionSpark") \
    .getOrCreate()

# Importamos las funciones de sql de pyspark para manejar mejor las columnas
from pyspark.sql.functions import col, sum, when, size, length, to_date, from_unixtime, year, count

from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, BooleanType

import importlib


In [2]:
from automatizacion_reviews import etl_reviews
from automatizacion_metadata import etl_metadata

In [3]:
df = etl_reviews('../datasets/Digital_Music.json')


In [4]:
df_meta = etl_metadata('../datasets/meta_Digital_Music.json')


In [5]:
df.show()

+----------+-------+--------------------+--------------+-----------------+--------+-------------------+-----------+
|      asin|overall|          reviewText|    reviewerID|     reviewerName|verified|        review_date|review_year|
+----------+-------+--------------------+--------------+-----------------+--------+-------------------+-----------+
|B000V6AEI6|    5.0|The composistions...| AAVK0CAPD9XH5|             mole|   false|2003-12-31 21:00:00|       2003|
|B000VRWV4U|    5.0|I needed some sca...|A3O39R049OZZUZ|            Wonka|    true|2012-12-31 21:00:00|       2012|
|B000TE3FBS|    1.0|never received it...| A19CD6KQG33SD|            Dixie|    true|2013-12-31 21:00:00|       2013|
|B0048W3SQ4|    5.0|WHAT ELSE CAN YOU...|A1WHQ7EFI717UH|   JOHNWAYNEFAN57|    true|2013-01-09 21:00:00|       2013|
|B0011W0BKA|    5.0|use this for our ...|A2JTT6E8TMNUIG|   Family of Five|    true|2013-01-10 21:00:00|       2013|
|B003G8WI7G|    5.0|Trombone Shorty's...|A3K9WWU19MSZ41|      CommonCent

In [6]:
df_ml = df.join(df_meta, 'asin', 'inner')

In [7]:
df_ml = df_ml.select(['asin','also_buy','also_view','brand','overall','price','reviewerID'])

In [8]:
df_ml.show(4)

+----------+--------------------+--------------------+-----------------+-------+-----+--------------+
|      asin|            also_buy|           also_view|            brand|overall|price|    reviewerID|
+----------+--------------------+--------------------+-----------------+-------+-----+--------------+
|B004CZ6APS|B0123VKB3O, B01LS...|B074JVBPJB, B0776...|          Nirvana|    5.0| 11.0|A19QEGW6UVO1BG|
|B000E6UXV6|                 N/D|B0142G2NM0, B0015...|Mrs. Brown's Boys|    5.0| 14.0| ACIACZMJ3ERHW|
|B000JF21D0|B009IRVSVQ, B003Z...|B009IRVSVQ, B003Z...|       Bruno Mars|    5.0| 9.99|A25FR6BAL3VM9E|
|B0002MEIDQ|B00078RPGW, B00EM...|          B00078RPGW|                 |    5.0| 9.99|A2DDBSQ9M5BEE9|
+----------+--------------------+--------------------+-----------------+-------+-----+--------------+
only showing top 4 rows



In [9]:
df_calificaciones = df_ml.select(['asin','reviewerID','overall'])

In [10]:
df_calificaciones.show(5)

+----------+--------------+-------+
|      asin|    reviewerID|overall|
+----------+--------------+-------+
|B004CZ6APS|A19QEGW6UVO1BG|    5.0|
|B000E6UXV6| ACIACZMJ3ERHW|    5.0|
|B000JF21D0|A25FR6BAL3VM9E|    5.0|
|B0002MEIDQ|A2DDBSQ9M5BEE9|    5.0|
|B000JF21D0|A1H8HIRHU8CJ3Q|    5.0|
+----------+--------------+-------+
only showing top 5 rows



In [11]:
from pyspark.ml.feature import StringIndexer

# Codificar la columna "usuario" en un nuevo DataFrame
indexer_usuario = StringIndexer(inputCol="reviewerID", outputCol="user_id")
df_calificaciones = indexer_usuario.fit(df_calificaciones).transform(df_calificaciones)

# Codificar la columna "producto" en un nuevo DataFrame
indexer_producto = StringIndexer(inputCol="asin", outputCol="product_id")
df_calificaciones = indexer_producto.fit(df_calificaciones).transform(df_calificaciones)

# Ahora el DataFrame df_calificaciones tiene columnas "usuario_index" y "producto_index" que son numéricas

In [12]:
df_calificaciones = df_calificaciones.select(['overall','user_id','product_id'])

In [13]:
df_calificaciones.show(4)

+-------+--------+----------+
|overall| user_id|product_id|
+-------+--------+----------+
|    5.0| 21939.0|     119.0|
|    5.0|107683.0|      53.0|
|    5.0| 48377.0|       1.0|
|    5.0|  8204.0|     129.0|
+-------+--------+----------+
only showing top 4 rows



In [14]:
from pyspark.ml.recommendation import ALS

# Crear una instancia del modelo ALS
als = ALS(
    maxIter=10,              # Número máximo de iteraciones (puedes ajustarlo)
    regParam=0.01,           # Parámetro de regularización (puedes ajustarlo)
    userCol="user_id",      # Nombre de la columna de usuario
    itemCol="product_id",     # Nombre de la columna de producto
    ratingCol="overall" # Nombre de la columna de calificación
)

# Entrenar el modelo ALS con los datos de calificación
modelo = als.fit(df_calificaciones)  # df_calificaciones es tu DataFrame de calificaciones

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "c:\Users\niko\Desktop\sistema_recomendacion_amazon\venv_sistema_amazon\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\niko\AppData\Local\Temp\ipykernel_1488\2406445277.py", line 13, in <module>
    modelo = als.fit(df_calificaciones)  # df_calificaciones es tu DataFrame de calificaciones
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\niko\Desktop\sistema_recomendacion_amazon\venv_sistema_amazon\Lib\site-packages\pyspark\ml\base.py", line 205, in fit
    return self._fit(dataset)
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\niko\Desktop\sistema_recomendacion_amazon\venv_sistema_amazon\Lib\site-packages\pyspark\ml\wrapper.py", line 381, in _fit
    java_model = self._fit_java(dataset)
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\niko\Desktop\sistema_recomendacion_amazon\venv_sist

ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión

In [ ]:
modelo

ALSModel: uid=ALS_93f682160dc0, rank=10

In [ ]:

usuario_id = 21939

usuario_df = spark.createDataFrame([(usuario_id,)], ["user_id"])


recomendaciones_usuario = modelo.recommendForUserSubset(usuario_df, numItems=10)  


In [ ]:
recomendaciones_usuario.printSchema()

root
 |-- user_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- product_id: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [ ]:
recomendaciones_usuario.show()

Py4JJavaError: An error occurred while calling o361.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 149.0 failed 1 times, most recent failure: Lost task 0.0 in stage 149.0 (TID 463) (niko executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RD